In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

### Specified stock

In [2]:
name = 'CHAYO'
sql = '''
SELECT name, year, aq_amt
FROM epss
WHERE name = '%s' AND quarter = 4
ORDER BY year'''
sql = sql % name
df = pd.read_sql(sql, conlt)
df

,name,year,aq_amt
0,CHAYO,2018,85459
1,CHAYO,2019,111427
2,CHAYO,2020,155354


In [3]:
df.pivot_table(index=['name'],columns='year',values='aq_amt')

year,2018,2019,2020
name,,,
CHAYO,85459,111427,155354


### All stocks

In [4]:
sql = '''
SELECT name, year, aq_amt
FROM epss
WHERE quarter = 4 AND year BETWEEN 2016 AND 2020
ORDER BY name, year DESC'''
df = pd.read_sql(sql, conlt)
df.aq_amt = df.aq_amt.astype(int)
df.head()

,name,year,aq_amt
0,2S,2020,410165
1,2S,2019,133382
2,2S,2018,105547
3,2S,2017,284707
4,2S,2016,284707


In [5]:
df_all = df.pivot_table(index=['name'],columns='year',values='aq_amt', margins=True,  # add margins
                        aggfunc=['max'],
                        fill_value=0)
df_all.head()

max                                                  
year        2016      2017      2018      2019      2020       All
name                                                              
2S        284707    284707    105547    133382    410165    410165
AAV      1869463   1477489     69965   -473999  -4764092   1869463
ACE            0         0         0    815306   1507641   1507641
ADVANC  30666538  30077312  29682178  31189571  27434360  31189571
AEONTS   2403464   2966331   3506487   3975357   3689643   3975357

### Portfolio stocks

In [6]:
sql = '''
SELECT name 
FROM buy
ORDER BY name'''
st_buy = pd.read_sql(sql, const)
names = st_buy.name
in_p = ', '.join(map(lambda name: "'%s'" % name, names))
in_p

"'BCH', 'BGRIM', 'CKP', 'DCC', 'DIF', 'DOHOME', 'EPG', 'GLOBAL', 'HREIT', 'IMH', 'IP', 'IVL', 'JASIF', 'KBANK', 'KCE', 'MCS', 'NER', 'NOBLE', 'PTT', 'PTTGC', 'RATCH', 'RCL', 'RJH', 'SAT', 'SCC', 'SIS', 'STA', 'TISCO', 'TMT', 'TOP', 'TSTH', 'TU', 'TYCN', 'VNT'"

In [7]:
sql = '''
SELECT name, year, aq_amt
FROM epss
WHERE quarter = 4 AND year BETWEEN 2016 AND 2020
AND name IN (%s)
ORDER BY name, year'''
sql = sql % in_p
df = pd.read_sql(sql, conlt)
df.shape

(154, 3)

In [8]:
df_pivot = df.pivot_table(index=['name'],
                          columns='year',
                          values='aq_amt',
                          margins=True,  # add margins
                          aggfunc='max',
                          fill_value=0)
df_pivot

year,2016,2017,2018,2019,2020,All
name,,,,,,
BCH,753093,917057,1089465,1134907,1229405,1229405
BGRIM,0,2126710,1862806,2331153,2174759,2331153
CKP,55055,158773,599068,768902,404714,768902
DCC,1421723,1116867,978755,972791,1585345,1585345
DIF,18867071,11115912,9854546,20992136,13424031,20992136
DOHOME,0,0,0,725511,726680,726680
EPG,1380978,990961,904318,1002796,1218890,1380978
GLOBAL,1494972,1608567,2003039,2093471,1955963,2093471
HREIT,0,-531737,650124,756360,609075,756360


In [9]:
df_pivot.describe

<bound method NDFrame.describe of year        2016       2017       2018      2019      2020        All
name                                                                 
BCH       753093     917057    1089465   1134907   1229405    1229405
BGRIM          0    2126710    1862806   2331153   2174759    2331153
CKP        55055     158773     599068    768902    404714     768902
DCC      1421723    1116867     978755    972791   1585345    1585345
DIF     18867071   11115912    9854546  20992136  13424031   20992136
DOHOME         0          0          0    725511    726680     726680
EPG      1380978     990961     904318   1002796   1218890    1380978
GLOBAL   1494972    1608567    2003039   2093471   1955963    2093471
HREIT          0    -531737     650124    756360    609075     756360
IMH            0          0          0     10499    -16607      10499
IP             0          0          0     43472     53776      53776
IVL     16197103   20882861   26465403   5252109   24142

In [10]:
file_name = "PIVOT-PORTFOLIO.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
data_file, output_file, box_file

df_pivot.to_csv(data_file, header=None)
df_pivot.to_csv(output_file)
df_pivot.to_csv(box_file)

### By market

In [11]:
sql = '''
SELECT name, market
FROM stocks
ORDER BY name'''
lt_stocks = pd.read_sql(sql, conlt)
lt_stocks.shape

(356, 2)

In [12]:
filters = [
   (lt_stocks.market.str.contains('SET50')),
   (lt_stocks.market.str.contains('SET100')),
   (lt_stocks.market.str.contains('mai'))    
]
values = ['SET50','SET100','mai']

In [13]:
lt_stocks["mrkt"] = np.select(filters, values, default='SET')
lt_stocks.head(4)

,name,market,mrkt
0,2S,SET,SET
1,AAV,SET100 / SETTHSI,SET100
2,ACE,SET100,SET100
3,ADVANC,SET50 / SETHD / SETTHSI,SET50


### Start of SET50

In [14]:
set50 = lt_stocks.mrkt.str.contains('SET50')
flt_set50 = lt_stocks[set50]
flt_set50.shape

(50, 3)

In [15]:
names = flt_set50.name
in_p = ', '.join(map(lambda name: "'%s'" % name, names))
in_p

"'ADVANC', 'AOT', 'BBL', 'BDMS', 'BEM', 'BGRIM', 'BH', 'BJC', 'BTS', 'CBG', 'COM7', 'CPALL', 'CPF', 'CPN', 'CRC', 'DELTA', 'DTAC', 'EA', 'EGCO', 'GLOBAL', 'GPSC', 'GULF', 'HMPRO', 'INTUCH', 'IRPC', 'IVL', 'KBANK', 'KCE', 'KTB', 'KTC', 'LH', 'MINT', 'MTC', 'OR', 'OSP', 'PTT', 'PTTEP', 'PTTGC', 'RATCH', 'SAWAD', 'SCB', 'SCC', 'SCGP', 'STA', 'STGT', 'TISCO', 'TOP', 'TRUE', 'TTB', 'TU'"

In [16]:
sql = '''
SELECT name, year, aq_amt
FROM epss
WHERE quarter = 4 AND year BETWEEN 2016 AND 2020
AND name IN (%s)
ORDER BY name, year'''
sql = sql % in_p
df_50 = pd.read_sql(sql, conlt)
df_50.shape

(229, 3)

In [17]:
df_merge = pd.merge(df_50,lt_stocks,on='name',how='inner')
df_merge.shape

(229, 5)

In [18]:
df_pivot_50 = df_merge.pivot_table(index=['mrkt','name'],
                          columns='year',
                          values='aq_amt',
                          margins=True,  # add margins
                          aggfunc='max',
                          fill_value=0)
df_pivot_50

year              2016       2017       2018      2019      2020        All
mrkt  name                                                                 
SET50 ADVANC  30666538   30077312   29682178  31189571  27434360   31189571
      AOT     19571456   20683603   25170761  25026369   4320675   25170761
      BBL     31814845   33008714   35329972  35816094  17180582   35816094
      BDMS     8386478   10215740    9191463  15517170   7214236   15517170
      BEM      2605854    3123129    5317038   5434819   2051090    5434819
      BGRIM          0    2126710    1862806   2331153   2174759    2331153
      BH       3626174    3943889    4151886   3747729   1204138    4151886
      BJC      4001190    5210755    6649971   7278382   4001215    7278382
      BTS      2003480    4415711    2872946   8161749   4576273    8161749
      CBG      1489756    1245812    1158572   2506461   3525028    3525028
      COM7      406545     608768     891063   1216323   1490682    1490682
      CPALL   16676510   19907708   20929650  22343085  16102417   22343085
      CPF     14702819   15259320   15531470  18455806  26022389   26022389
      CPN      9243797   13567645   11215653  11738405   9557097   13567645
      CRC            0          0          0  10931181     46266   10931181
      DELTA    5516290    4930976    5137029   2959960   7101638    7101638
      DTAC           0          0   -4368692   5421893   5107119    5421893
      EA       3251506    3817450    4975211   6081619   5204566    6081619
      EGCO     8320801   11818281   21072883  13059236   8733031   21072883
      GLOBAL   1494972    1608567    2003039   2093471   1955963    2093471
      GPSC     2699903    3174581    3359187   4060797   7508126    7508126
      GULF           0    3451424    3028135   4886564   4282114    4886564
      HMPRO    4125196    4886387    5612624   6176590   5154702    6176590
      INTUCH  16397609   10673179   11490679  11083014  11047654   16397609
      IRPC     9720831   11354484    7734992  -1174044  -6151700   11354484
      IVL     16197103   20882861   26465403   5252109   2414276   26465403
      KBANK   40174100   34338246   38459116  38726742  29487117   40174100
      KCE      3038747    2544502    2014872    934492   1126787    3038747
      KTB     32278096   22445401   28491290  29284042  16731585   32278096
      KTC      2494705    3304324    5139589   5524072   5332865    5524072
      LH       8617974   10463217   10475429  10024906   7144917   10475429
      MINT     6589996    5415397    5444771  10697927 -21407336   10697927
      MTC      1464138    2500598    3713385   4237469   5213924    5213924
      OR             0          0          0         0   8791070    8791070
      OSP            0          0    3005343   3259009   3504313    3504313
      PTT     94609078  135179603  119683943  92950604  37765808  135179603
      PTTEP   12859719   20579465   36206290  48802538  22664011   48802538
      PTTGC   25601633   39298310   40069490  11682080    199607   40069490
      RATCH    6165722    6106700    5587602   5963281   6286678    6286678
      SAWAD    2004623    2666599    2768360   3756487   4508329    4508329
      SCB     47612227   43151904   40067556  40436350  27217602   47612227
      SCC     56084194   55041247   44748337  32014283  34143870   56084194
      SCGP           0          0          0         0   6457475    6457475
      STA      -757986   -1437051    2064358   -148536   9531213    9531213
      STGT           0          0          0         0  14400869   14400869
      TISCO    5005402    6090011    7015685   7270228   6063483    7270228
      TOP     21221911   24856199   10149038   6276682  -3301407   24856199
      TRUE    -2814348    2322526    7034590   5636733   1048398    7034590
      TTB      8226258    8686655   11601235   7222478  10112182   11601235
      TU       5254432    6020737    3256211   3815877   6246090    6246090
All           94609078  135179603  119683943  92

### End of SET50

In [19]:
set100 = lt_stocks.mrkt.str.contains('SET100')
flt_set100 = lt_stocks[set100]
flt_set100.shape

(50, 3)

In [20]:
names = flt_set100.name
in_p = ', '.join(map(lambda name: "'%s'" % name, names))
in_p

"'AAV', 'ACE', 'AEONTS', 'AMATA', 'AP', 'BAM', 'BANPU', 'BCH', 'BCP', 'BCPG', 'BEC', 'CENTEL', 'CHG', 'CK', 'CKP', 'DOHOME', 'ESSO', 'GUNKUL', 'HANA', 'ICHI', 'JAS', 'JMART', 'JMT', 'KKP', 'MAJOR', 'MEGA', 'NRF', 'ORI', 'PLANB', 'PRM', 'PSL', 'PTG', 'PTL', 'QH', 'RBF', 'RS', 'SINGER', 'SPALI', 'SPRC', 'STEC', 'SUPER', 'SYNEX', 'TASCO', 'TCAP', 'THANI', 'TKN', 'TQM', 'TVO', 'VGI', 'WHA'"

In [21]:
sql = '''
SELECT name, year, aq_amt
FROM epss
WHERE quarter = 4 AND year BETWEEN 2016 AND 2020
AND name IN (%s)
ORDER BY name, year'''
sql = sql % in_p
df_100 = pd.read_sql(sql, conlt)
df_100.shape

(223, 3)

In [22]:
df_merge = pd.merge(df_100,lt_stocks,on='name',how='inner')
df_merge.shape

(223, 5)

In [23]:
df_pivot_100 = df_merge.pivot_table(index=['mrkt','name'],
                          columns='year',
                          values='aq_amt',
                          margins=True,  # add margins
                          aggfunc='max',
                          fill_value=0)
df_pivot_100

year              2016     2017     2018      2019     2020       All
mrkt   name                                                          
SET100 AAV     1869463  1477489    69965   -473999 -4764092   1869463
       ACE           0        0        0    815306  1507641   1507641
       AEONTS  2403464  2966331  3506487   3975357  3689643   3975357
       AMATA   1198273  1409494  1018223   1742056  1103439   1742056
       AP      2702561  3157096  3865410   3067513  4226535   4226535
       BAM           0        0        0   6549300  1840622   6549300
       BANPU   1677115  7900243  6693731   -552864 -1786317   7900243
       BCH      753093   917057  1089465   1134907  1229405   1229405
       BCP     4773384  5778239  2462881   1731572 -6967070   5778239
       BCPG    1541325  2016151  2219228   1801424  1912252   2219228
       BEC           0        0        0   -397166  -214254   -214254
       CENTEL  1849555  1991390  2177612   1744243 -2775115   2177612
       CHG      564285   565444   633879    705236   876617    876617
       CK      2002402  1810007  2494309   1777782   612158   2494309
       CKP       55055   158773   599068    768902   404714    768902
       DOHOME        0        0        0    725511   726680    726680
       ESSO    6648754  7287621  2225164  -3065703 -7911074   7287621
       GUNKUL   537717    22061  1085172   2147329  3412336   3412336
       HANA    2105419  2887731  2375223   1804584  1907741   2887731
       ICHI     368481   315094    43841    407451   515533    515533
       JAS     3001605  2693393  4912591   7264608 -3125750   7264608
       JMART         0        0        0    533849   797871    797871
       JMT      290407   396132   505516    681363  1047041   1047041
       KKP     5546725  5736869  6041983   5988444  5123266   6041983
       MAJOR   1188220  1193337  1283590   1170028   -57493   1283590
       MEGA     794909  1112797  1206008   1138515  1392651   1392651
       NRF           0        0        0         0   124391    124391
       ORI      637563  2020883  3337953   3027133  2661894   3337953
       PLANB    351600   460516   642778    743195   140234    743195
       PRM           0   717932   711821   1023385  1533057   1533057
       PSL    -2664899  -129476   456203   -228491 -1294855    456203
       PTG     1073447   913098   624742   1560676  1894041   1894041
       PTL     1366361  1169985  2324387   1803077  2970004   2970004
       QH      3084974  3462236  3800740   2853861  2123420   3800740
       RBF           0        0        0    352777   519016    519016
       RS      -102145   332862   516040    363343   528278    528278
       SINGER        0        0   -80767    165894   443298    443298
       SPALI   4886527  5812046  5770419   5402758  4251230   5812046
       SPRC    8688088  8895346  2263254  -2808704 -6004841   8895346
       STEC    1380755  -610826  1616863   1483636  1093135   1616863
       SUPER    361624  1284319  1045433   2137108  1303278   2137108
       SYNEX    406932   623840   721491    523933   641951    721491
       TASCO   3110134  2537111   564466   3123280  3591824   3591824
       TCAP    6012725  7001170  7839428  10807059  6669030  10807059
       THANI    881157  1125813  1640755   1963564  1860203   1963564
       TKN      781848   608437   459181    366238   242621    781848
       TQM           0        0   404266    507230   701995    701995
       TVO     2754625  1326759  1867437   1410529  1655800   2754625
       VGI      826402   846226  1101243   1423940   979765   1423940
       WHA     2898156  3266429  2906810   3229254  2523750   3266429
All            8688088  8895346  7839428  10807059  6669030  10807059

### End of SET100

In [24]:
setmai = lt_stocks.mrkt.str.contains('mai')
flt_mai = lt_stocks[setmai]
flt_mai.shape

(15, 3)

In [25]:
names = flt_mai.name
in_p = ', '.join(map(lambda name: "'%s'" % name, names))
in_p

"'ARROW', 'FPI', 'FSMART', 'IMH', 'INSET', 'IP', 'JUBILE', 'LIT', 'MBAX', 'PDG', 'SMART', 'TACC', 'TMW', 'TQR', 'YUASA'"

In [26]:
sql = '''
SELECT name, year, aq_amt
FROM epss
WHERE quarter = 4 AND year BETWEEN 2016 AND 2020
AND name IN (%s)
ORDER BY name, year'''
sql = sql % in_p
df_mai = pd.read_sql(sql, conlt)
df_mai.shape

(54, 3)

In [27]:
df_merge = pd.merge(df_mai,lt_stocks,on='name',how='inner')
df_merge.shape

(54, 5)

In [28]:
df_pivot_mai = df_merge.pivot_table(index=['mrkt','name'],
                          columns='year',
                          values='aq_amt',
                          margins=True,  # add margins
                          aggfunc='max',
                          fill_value=0)
df_pivot_mai

year           2016    2017    2018    2019    2020     All
mrkt name                                                  
mai  ARROW   262708  191302  153740  205470  168984  262708
     FPI     285671  204230  125550  186790   18747  285671
     FSMART  420333  542896  583105  584226  464063  584226
     IMH          0       0       0   10499  -16607   10499
     INSET        0       0       0       0  135491  135491
     IP           0       0       0   43472   53776   53776
     JUBILE  157281  184718  191190  262209  266789  266789
     LIT     100662  145489  148846  103098   71241  148846
     MBAX         0       0       0   50977  176063  176063
     PDG      93329   74412   56806   65878   98255   98255
     SMART        0       0       0   44235   45807   45807
     TACC    101830  112473   68393  162091  188381  188381
     TMW     174884  333955  370613  235648   60836  370613
     YUASA   113731  111460  128202  138920  158177  158177
All          420333  542896  583105  584226  464063  584226

### End of mai

In [29]:
flt_999 = lt_stocks[~(set50 | set100 | setmai)] 
flt_999.shape

(241, 3)

In [30]:
names = flt_999.name
in_p = ', '.join(map(lambda name: "'%s'" % name, names))
in_p

"'2S', 'AH', 'AIE', 'AIMIRT', 'AIT', 'AJ', 'AKR', 'ALUCON', 'AMANAH', 'AMARIN', 'AMC', 'ANAN', 'APCS', 'APURE', 'ASIAN', 'ASK', 'ASP', 'ASW', 'AWC', 'AYUD', 'BA', 'BAFS', 'BAY', 'BCT', 'BEAUTY', 'BEYOND', 'BFIT', 'BGC', 'BKI', 'BLA', 'BLAND', 'BPP', 'BRR', 'BSBM', 'BTSGIF', 'CCET', 'CGH', 'CHAYO', 'CIMBT', 'COTTO', 'CPI', 'CPNCG', 'CPNREIT', 'CPTGF', 'CSC', 'CSP', 'CSS', 'CWT', 'DCC', 'DCON', 'DDD', 'DEMCO', 'DIF', 'DRT', 'DUSIT', 'EASTW', 'ECL', 'EGATIF', 'EKH', 'EMC', 'EPG', 'FORTH', 'FSS', 'GBX', 'GC', 'GFPT', 'GGC', 'GLAND', 'GPI', 'GRAMMY', 'GVREIT', 'HFT', 'HREIT', 'HTC', 'HTECH', 'HUMAN', 'ICC', 'IFS', 'IHL', 'III', 'ILINK', 'ILM', 'IMPACT', 'INET', 'INGRS', 'INOX', 'IRC', 'J', 'JASIF', 'JTS', 'JWD', 'KAMART', 'KGI', 'KSL', 'KYE', 'LALIN', 'LANNA', 'LHFG', 'LHHOTEL', 'LHK', 'LHSC', 'LPF', 'LPH', 'LPN', 'LST', 'M', 'MACO', 'MAKRO', 'MC', 'MCS', 'MDX', 'MFC', 'MFEC', 'MSC', 'MST', 'MTI', 'NCH', 'NER', 'NNCL', 'NOBLE', 'NTV', 'NYT', 'OISHI', 'PAP', 'PB', 'PCSGH', 'PDJ', 'PERM', 'PL

In [31]:
sql = '''
SELECT name, year, aq_amt
FROM epss
WHERE quarter = 4 AND year BETWEEN 2016 AND 2020
AND name IN (%s)
ORDER BY name, year'''
sql = sql % in_p
df_999 = pd.read_sql(sql, conlt)
df_999.shape

(1043, 3)

In [32]:
df_merge = pd.merge(df_999,lt_stocks,on='name',how='inner')
df_merge.shape

(1043, 5)

In [33]:
pd.set_option('max_rows',None)
df_pivot_999 = df_merge.pivot_table(index=['mrkt','name'],
                          columns = 'year',
                          values ='aq_amt',
                          margins = True,  # add margins
                          aggfunc ='max',
                          fill_value = 0)
df_pivot_999

year               2016      2017      2018      2019      2020       All
mrkt name                                                                
SET  2S          284707    284707    105547    133382    410165    410165
     AH          543092   1157614   1299159   -181108    147764   1299159
     AIE              0         0         0   -156496    488517    488517
     AIMIRT           0         0    290715    386346    413822    413822
     AIT         429392    431130    236834    392093    394271    431130
     AJ          121253    443443     17983    236168    532252    532252
     AKR              0         0         0     82545     98443     98443
     ALUCON     1069812    783739    719666    532116    669003   1069812
     AMANAH       31436     47520    163594    247539    290708    290708
     AMARIN     -628116   -163937    172681    167717    170669    172681
     AMC              0         0         0    145763    133051    145763
     ANAN       1501442   1328454   3419427    704658   -206577   3419427
     APCS         55599    148368    221457    238743     14072    238743
     APURE       191597    141817    119548     29141    319617    319617
     ASIAN       154635    418022    362235    132697    818056    818056
     ASK         706684    745043    811289    869534    883064    883064
     ASP         755646    743421    434200    357936    415777    755646
     AWC              0         0         0   1054229  -1881229   1054229
     AYUD        615165    462910    750248    189628    631114    750248
     BA         1768408    787912    249263    350834  -5283182   1768408
     BAFS       1097117    970587   1050587    939922   -374261   1097117
     BAY       21404034  23209271  24812637  32748508  23039785  32748508
     BCT        2292660   2281819   2584873   2394264   1036420   2584873
     BEAUTY      656006   1229318    991593    232577   -104882   1229318
     BEYOND           0         0         0     -5764     80435     80435
     BFIT        129816    234809    369425    782504   1502997   1502997
     BGC              0         0    496170    511616    516411    516411
     BKI        2425788   2403097   2406808   2451281   2705596   2705596
     BLA        5122365   3635388   4969739   4385986   1608268   5122365
     BLAND      1848557   1230913   1815821   1216596   -801332   1848557
     BPP        4138126   4154634   3812668   2968932   3702480   4154634
     BRR         113083    524726    271621   -511711      6138    524726
     BSBM        268102    131894     33112     48549     84017    268102
     BTSGIF     4849474   4221555  -1774651  -1999838  -5834605   4849474
     CCET       1168564    961728    321643    392799    209607   1168564
     CGH              0         0         0     82610    142178    142178
     CHAYO            0         0     85459    111427    155354    155354
     CIMBT      -629525    384898      6901   1501596   1290582   1501596
     COTTO            0         0         0         0    420190    420190
     CPI              0         0         0     90136    159055    159055
     CPNCG       243797    249232    402056    433157    501658    501658
     CPNREIT    3162669   -407328   3826555   3322938    304596   3826555
     CPTGF            0         0    866667    902125    524188    902125
     CSC         372436    249764    330194    348980    258434    372436
     CSP              0         0         0   -127445     84304     84304
     CSS         169578    203905    170338     79649     92701    203905
     CWT          43600     52833     74162    105262    117031    117031
     DCC        1421723   1116867    978755    972791   1585345   1585345
     DCON        138144    223306    163105    131614     92434    223306
     DDD              0    351063    181409    -53785    169253    351063
     DEMCO      -160772     65585    148602     34962     55201    148602
     DIF       18867071  11115912   9854546  20992136  13424031  20992136
    

In [34]:
file_name = "df_pivot_999.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
data_file, output_file, box_file

df_pivot_999.to_csv(data_file, header=None)
df_pivot_999.to_csv(output_file)
df_pivot_999.to_csv(box_file)